<a href="https://colab.research.google.com/github/sathwikkes/Yelp-Image-Classification/blob/main/data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# I will be working with Yelp Image Data set

In [2]:
#!pip list -v 
#checking/making sure I have the proper modules loaded

In [3]:
# importing necessary modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os 
import io
import seaborn as sns

In [4]:
from google.colab import files
uploaded = files.upload()

Saving photos.json to photos (3).json


In [13]:
type(uploaded)

dict

In [12]:
df = pd.read_csv(io.StringIO(uploaded.get("photos.json").decode('UTF-8')))

ParserError: ignored

In [6]:
data = next(iter(uploaded.values()))

In [7]:
d = json.loads(data.decode())

JSONDecodeError: ignored

In [9]:
try:
	a_json = json.loads(data.decode('UTF-8'))
	print(a_json)
except json.decoder.JSONDecodeError:
	print("String could not be converted to JSON")

String could not be converted to JSON


In [12]:
#file_name="photos.json"
#io.StringIO[file_name].decode("utf-8")

TypeError: ignored

In [6]:
#json.loads(uploaded[file_name].decode('utf-8'))

JSONDecodeError: ignored

In [7]:
#photo_json = "/Users/sathwikthecreator/Desktop/projects/image-classification/data/yelp_photos/photos.json"

In [9]:
#with open(photo_json) as f: 
#  data = json.load(f)
#print(data)

FileNotFoundError: ignored